# Analysis of Siren based trained models vs JPEG

- enable dark mode jupiter:
    - jt -t monokai -f fira -fs 10 -nf ptsans -nfs 11 -N -kl -cursw 2 -cursc r -cellw 95% -T
- disable set theme:
    - jt -r

### Fetch Libs, data

In [ ]:
# --- Import libs for running notebook.

from utils.libs import *

In [ ]:
# --- Read config file for runnig notebook.

conf_file_path = 'conf.txt'
conf_data = read_conf_file(conf_file_path)

# pprint(conf_data)

In [ ]:
# --- Fetch data about carryed out trainings.
train_df, _, records_list = fetch_data(conf_data)

In [ ]:
# np.unique(pd.DataFrame(data = list(map(operator.methodcaller('_asdict'), records_list)))['image_size'].values)
if records_list is not None:
    records_df = pd.DataFrame(data = list(map(operator.methodcaller('_asdict'), records_list)))
    print(records_df.info())
    print(records_df.sort_values(by = ['hidden_features']).head(5))
    pass

In [ ]:
train_df.head(5)

In [ ]:
train_df.info()

In [ ]:
train_df.describe()

### Calculate Jpge

In [ ]:
# --- Load target image.
im = load_target_image(image_file_path = None)
print('Image size:', im.size)
im

In [ ]:
crop_size = conf_data['cropped_image']['crop_size']
if isinstance(crop_size, str):
    crop_size = eval(crop_size)
elif isinstance(crop_size, int):
    crop_size = (crop_size, crop_size)
    pass
# type(crop_size[0])

In [ ]:
if conf_data['cropped_image']['flag'] is True:
    im_cropped = get_cropped_by_center_image(im, target = crop_size)
    pass
else:
    im_cropped = im
print('Image size:', im_cropped.size)
im_cropped

In [ ]:
# --- Array of qualities to be tested in compression.
qualities_arr = np.arange(20, 95+1, dtype = np.int)

In [ ]:
# --- Run several trials for JPEG compression.

cropped_file_size_bits = None

# im_cropped = im
# im_cropped = Image.fromarray(skimage.data.camera())
with BytesIO() as f:
    im_cropped.save(f, format='PNG')
    cropped_file_size_bits = f.getbuffer().nbytes * 8
    pass

result_tuples, failure_qualities = \
  calculate_several_jpeg_compression(im_cropped, cropped_file_size_bits, qualities_arr)

In [ ]:
# data = list(map(lambda xx: xx._asdict(), result_tuples))
data = list(map(operator.methodcaller('_asdict'), result_tuples))
result_jpeg_df = pd.DataFrame(data = data)
result_jpeg_df.head(5)

### Show graphics.

In [ ]:
# sns.pairplot(results_history_df)
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    try:
        g = sns.PairGrid(result_jpeg_df.drop(['width', 'heigth'], axis = 1), diag_sharey=False)
        g.map_upper(sns.scatterplot, s=15) # 
        g.map_lower(sns.kdeplot)
        g.map_diag(sns.kdeplot, lw=2)
        # plt.savefig(f"{images_conf.df_jpeg_scatterplot}")
    except Exception as err:
        print(str(err))
        pass
    pass

In [ ]:
train_df['bpp'] = train_df['#params'].values * 32 / (im_cropped.size[0] * im_cropped.size[1])

In [ ]:
compare_compressions(compression_1_df=train_df, compression_2_df=result_jpeg_df)

In [ ]:
result_jpeg_df.columns, train_df.columns

In [ ]:
train_df['compression'] = list(map(lambda hf: f'siren-{hf}', train_df['hf'].values.astype(dtype = np.int)))

In [ ]:
result_jpeg_df['compression'] = ['jpeg'] * result_jpeg_df.shape[0]
# train_df['compression'] = ['siren'] * train_df.shape[0]
train_df['file_size_bits'] = train_df['#params'].values * 32
train_df['CR'] = cropped_file_size_bits / (train_df['#params'].values * 32)

In [ ]:
result_jpeg_df.columns, train_df.columns

In [ ]:
train_df_columns_for_merge = "psnr,ssim,CR,bpp,file_size_bits,compression".split(",")
result_jpeg_df_columns_for_merge = "psnr,ssim,CR,bpp,file_size_bits,compression".split(",")

data_frames_list = [
    train_df[train_df_columns_for_merge],
    result_jpeg_df[result_jpeg_df_columns_for_merge],
]
merged_df = pd.concat(data_frames_list, names = "psnr,ssim,CR,bpp,file_size_bits,compression".split(","))

In [ ]:
g = sns.PairGrid(merged_df, hue="compression")
g.map_diag(plt.hist)
# g.map_offdiag(sns.scatterplot, marker='x')
g.map_upper(sns.scatterplot, marker='x', s=15) 
g.map_lower(sns.kdeplot)
g.add_legend()
# g.savefig("scatter_plot_dataframes_merged.png")
plt.show()

In [ ]:
x = 'bpp'
x_vars = "bpp,file_size_bits".split(",")
y_vars = "psnr,ssim,CR".split(",")[0:1]
y = "psnr"
hue = "compression"

fig, ax = plt.subplots(1, len(y_vars[0:1]), figsize = (10, 10))
# fig.suptitle(f"{x} vs. others")

chart = sns.scatterplot(x=f"{x}", y=f"{y}", data=merged_df, ax=ax, hue='compression', marker='x')
ax.set_title(f'{y} vs. {x}')
# chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

plt.show()

In [ ]:
x = 'bpp'
x_vars = "bpp,file_size_bits".split(",")
y_vars = "psnr,ssim,CR".split(",")[0:1]
y = "ssim"
hue = "compression"

fig, ax = plt.subplots(1, len(y_vars[0:1]), figsize = (10, 10))
# fig.suptitle(f"{x} vs. others")

chart = sns.scatterplot(x=f"{x}", y=f"{y}", data=merged_df, ax=ax, hue='compression', marker='x')
ax.set_title(f'{y} vs. {x}')
# chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

plt.show()

In [ ]:
x = 'bpp'
x_vars = "bpp,file_size_bits".split(",")
y_vars = "psnr,ssim,CR".split(",")[0:1]
y = "CR"
hue = "compression"

fig, ax = plt.subplots(1, len(y_vars[0:1]), figsize = (10, 10))
# fig.suptitle(f"{x} vs. others")

sns.scatterplot(x=f"{x}", y=f"{y}", data=merged_df, ax=ax, hue='compression', marker='x')
ax.set_title(f'{y} vs. {x}')
# chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')

plt.show()

In [ ]:
x = 'bpp'
x_vars = "bpp,file_size_bits".split(",")
y_vars = "psnr,ssim,CR".split(",")
hue = "compression"

fig, axes = plt.subplots(1, len(y_vars), figsize = (20, 5))
fig.suptitle(f"{x} vs. others")

for ax, y in zip(axes, y_vars):
    chart = sns.scatterplot(x=f"{x}", y=f"{y}", data=merged_df, ax=ax, hue='compression', marker='x')
    ax.set_title(f'{y} vs. {x}')
    chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')
    pass
plt.show()

In [ ]:
x = 'compression'

fig, axes = plt.subplots(1, len(y_vars), figsize = (20, 5))
fig.suptitle(f"{x} vs. others")

for ax, y in zip(axes, y_vars):
    chart = sns.boxplot(x=f"{x}", y=f"{y}", data=merged_df, ax=ax)
    ax.set_title(f'{y} vs. {x}')
    chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')
    pass
plt.show()

In [ ]:
x_vars = "bpp,file_size_bits".split(",")
y_vars = "psnr,ssim,CR".split(",")
hue = "compression"

sns.boxplot(x="compression", y="psnr", data=merged_df)

plt.show()

In [ ]:
x_vars = "bpp,file_size_bits".split(",")
y_vars = "psnr,ssim,CR".split(",")
hue = "compression"

sns.boxplot(x="compression", y="ssim", data=merged_df)

plt.show()

In [ ]:
x_vars = "bpp,file_size_bits".split(",")
y_vars = "psnr,ssim,CR".split(",")
hue = "compression"

sns.boxplot(x="compression", y="CR", data=merged_df)

plt.show()

In [ ]:
x_vars = "bpp,file_size_bits".split(",")
y_vars = "psnr,ssim,CR".split(",")
hue = "compression"

g = sns.PairGrid(merged_df, x_vars=x_vars, y_vars=y_vars, hue=hue)
g.map(sns.scatterplot, marker='x')
g.add_legend()

# g.savefig("compare_compressions_metrices.png")

plt.show()

In [ ]:
x = 'file_size_bits'
x_vars = "bpp,file_size_bits".split(",")
y_vars = "psnr,ssim,CR".split(",")
hue = "compression"

fig.suptitle(f"{x} vs. others")

for ax, y in zip(axes, y_vars):
    chart = sns.scatterplot(x=f"{x}", y=f"{y}", data=merged_df, ax=ax, hue='compression')
    ax.set_title(f'{y} vs. {x}')
    chart.set_xticklabels(chart.get_xticklabels(), rotation=45, horizontalalignment='right')
    pass
plt.show()

In [ ]:
markers =  (["x"] * (len(set(merged_df["compression"].values)) - 1)) + ["o"]

In [ ]:
# sns.__version__

In [ ]:
# ?sns.jointplot

In [ ]:
# sns.jointplot(data=merged_df, x="bpp", y="psnr", hue="compression")

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    g = (sns.jointplot("file_size_bits", "psnr",
            kind="reg",
            data=merged_df,
            color="b",
            # annot_kws=dict(stat="r"),
            stat_func=scipy.stats.kendalltau,)
        .plot_joint(sns.kdeplot, zorder=0, n_levels=6)
        .plot_joint(sns.regplot, color="b"))
    g.plot_marginals(sns.rugplot, color="r", height=-.15, clip_on=False)
    g.ax_marg_x.set_xscale('log')
    g.ax_marg_y.set_yscale('log')
    pass

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    g = (sns.jointplot("bpp", "psnr",
        kind="reg",
        data=merged_df, color="b", stat_func=scipy.stats.kendalltau,)
        .plot_joint(sns.kdeplot, zorder=0, n_levels=6)
        .plot_joint(sns.regplot, color="b"))
    g.plot_marginals(sns.rugplot, color="r", height=-.15, clip_on=False)
    g.ax_marg_x.set_xscale('log')
    g.ax_marg_y.set_yscale('log')
    pass

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    g = (sns.jointplot("file_size_bits", "ssim",
        kind="reg",
        data=merged_df, color="b", stat_func=scipy.stats.kendalltau,)
        .plot_joint(sns.kdeplot, zorder=0, n_levels=6)
        .plot_joint(sns.regplot, color="b"))
    g.plot_marginals(sns.rugplot, color="r", height=-.15, clip_on=False)
    g.ax_marg_x.set_xscale('log')
    g.ax_marg_y.set_yscale('log')
    pass

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    g = (sns.jointplot("bpp", "ssim",
        kind="reg",
        data=merged_df, color="b", stat_func=scipy.stats.kendalltau,)
        .plot_joint(sns.kdeplot, zorder=0, n_levels=6)
        .plot_joint(sns.regplot, color="b"))
    g.plot_marginals(sns.rugplot, color="r", height=-.15, clip_on=False)
    g.ax_marg_x.set_xscale('log')
    g.ax_marg_y.set_yscale('log')
    pass

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    g = (sns.jointplot("file_size_bits", "CR",
        kind="reg",
        data=merged_df, color="b", stat_func=scipy.stats.kendalltau,)
        .plot_joint(sns.kdeplot, zorder=0, n_levels=6)
        .plot_joint(sns.regplot, color="b"))
    g.plot_marginals(sns.rugplot, color="r", height=-.15, clip_on=False)
    g.ax_marg_x.set_xscale('log')
    g.ax_marg_y.set_yscale('log')
    pass

In [ ]:
with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    g = (sns.jointplot("bpp", "CR",
        kind="reg",
        data=merged_df, color="b", stat_func=scipy.stats.kendalltau,)
        .plot_joint(sns.kdeplot, zorder=0, n_levels=6)
        .plot_joint(sns.regplot, color="b"))
    g.plot_marginals(sns.rugplot, color="r", height=-.15, clip_on=False)
    g.ax_marg_x.set_xscale('log')
    g.ax_marg_y.set_yscale('log')
    pass

In [ ]:
# Plot sepal width as a function of sepal_length across days
g = sns.lmplot(
    data=merged_df,
    x="bpp", y="psnr", hue="compression",
    height=5,
    palette="Set1", markers=markers
).fig.subplots_adjust(wspace=.02)

In [ ]:
# Plot sepal width as a function of sepal_length across days
g = sns.lmplot(
    data=merged_df,
    x="file_size_bits", y="psnr", hue="compression",
    height=5,
    palette="Set1", markers=markers).fig.subplots_adjust(wspace=.02)

# Plot sepal width as a function of sepal_length across days
g = sns.lmplot(
    data=merged_df,
    x="file_size_bits", y="ssim", hue="compression",
    height=5,
    palette="Set1", markers=markers).fig.subplots_adjust(wspace=.02)
plt.show()

# Plot sepal width as a function of sepal_length across days
g = sns.lmplot(
    data=merged_df,
    x="file_size_bits", y="CR", hue="compression",
    height=5,
    palette="Set1", markers=markers).fig.subplots_adjust(wspace=.02)
plt.show()

In [ ]:
# Plot sepal width as a function of sepal_length across days
g = sns.lmplot(
    data=merged_df,
    x="bpp", y="psnr", hue="compression",
    height=5,
    palette="Set1", markers=markers).fig.subplots_adjust(wspace=.02)

# Plot sepal width as a function of sepal_length across days
g = sns.lmplot(
    data=merged_df,
    x="bpp", y="ssim", hue="compression",
    height=5,
    palette="Set1", markers=markers).fig.subplots_adjust(wspace=.02)
plt.show()

# Plot sepal width as a function of sepal_length across days
g = sns.lmplot(
    data=merged_df,
    x="bpp", y="CR", hue="compression",
    height=5,
    palette="Set1", markers=markers).fig.subplots_adjust(wspace=.02)
plt.show()

In [ ]:
g = sns.lmplot(x="bpp", y="psnr", col="compression", data=merged_df, palette="Set1").fig.subplots_adjust(wspace=.02)

In [ ]:
# Draw a pointplot to show pulse as a function of three categorical factors
g = sns.catplot(x="bpp", y="psnr", col="compression",
                capsize=.2, palette="YlGnBu_d", height=6, aspect=.75,
                kind="point", data=merged_df)
g.despine(left=True)
plt.show()

In [ ]:
# Draw a pointplot to show pulse as a function of three categorical factors
g = sns.catplot(x="bpp", y="ssim", col="compression",
                capsize=.2, palette="YlGnBu_d", height=6, aspect=.75,
                kind="point", data=merged_df)
g.despine(left=True)
plt.show()

In [ ]:
# Draw a pointplot to show pulse as a function of three categorical factors
g = sns.catplot(x="bpp", y="CR", col="compression",
                capsize=.2, palette="YlGnBu_d", height=6, aspect=.75,
                kind="point", data=merged_df)
g.despine(left=True)
plt.show()

### References:
---
#### Standard Libs:
  - [Lib/statistics.py](https://docs.python.org/3/library/statistics.html)
---
#### Seaborn:
  - [seaborn.jointplot](http://seaborn.pydata.org/generated/seaborn.jointplot.html?highlight=reg)
---
#### Drawing from data:
  - [drawingfromdata](https://www.drawingfromdata.com/)
  - [HOW TO ROTATE AXIS LABELS IN SEABORN AND MATPLOTLIB](https://www.drawingfromdata.com/how-to-rotate-axis-labels-in-seaborn-and-matplotlib)